In [6]:
import os, tabulate
print(tabulate.tabulate(sorted([(key, item) for key, item in os.environ.items()], key=lambda t: t[0])))

--------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------
AZ_AD_ID                          8f0f2649-5bd6-4956-aae2-81774d4804d1
AZ_AD_PASS                        fejredf45_^^
AZ_AD_TENANT                      bb482d8d-d073-4641-8d00-61ee67178c8c
CLICOLOR                          1
CLUTTER_IM_MODULE                 xim
COMPIZ_CONFIG_PROFILE             ubuntu
DBUS_SESSION_BUS_ADDRESS          unix:abstract=/tmp/dbus-QP4Pjjnrvf
DEFAULTS_PATH                     /usr/share/gconf/ubuntu.default.path
DESKTOP_SESSION                   ubuntu
DISPLAY                           :0
GDMSESSION                        ubuntu
GDM_LANG                          en_US
GIT_PAGER                         cat
GNOME_DESKTOP_SESSION_ID          this-is-deprecated
GNOME_KEYRING_CONTROL
GNOME_KEYRING_PID
GPG_AGENT_INFO                    /home/orpington/.gnupg/S.gpg-agent:0:

In [1]:
import os
import sys

os.environ["SPARK_HOME"] = "/opt/spark"
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib'))

In [2]:
from pyspark.sql import SparkSession
session = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
data = session.read.format("csv").option("sep", "|").load("/home/orpington/dbgen-data/lineitem.tbl")

In [34]:
data.show(10)

+---+------+----+---+---+--------+----+----+---+---+----------+----------+----------+-----------------+-------+--------------------+----+
|_c0|   _c1| _c2|_c3|_c4|     _c5| _c6| _c7|_c8|_c9|      _c10|      _c11|      _c12|             _c13|   _c14|                _c15|_c16|
+---+------+----+---+---+--------+----+----+---+---+----------+----------+----------+-----------------+-------+--------------------+----+
|  1|155190|7706|  1| 17|21168.23|0.04|0.02|  N|  O|1996-03-13|1996-02-12|1996-03-22|DELIVER IN PERSON|  TRUCK|egular courts abo...|null|
|  1| 67310|7311|  2| 36|45983.16|0.09|0.06|  N|  O|1996-04-12|1996-02-28|1996-04-20| TAKE BACK RETURN|   MAIL|ly final dependen...|null|
|  1| 63700|3701|  3|  8|13309.60|0.10|0.02|  N|  O|1996-01-29|1996-03-05|1996-01-31| TAKE BACK RETURN|REG AIR|riously. regular,...|null|
|  1|  2132|4633|  4| 28|28955.64|0.09|0.06|  N|  O|1996-04-21|1996-03-30|1996-05-16|             NONE|    AIR|lites. fluffily e...|null|
|  1| 24027|1534|  5| 24|22824.48|

In [2]:
raw_schema = """
	[L_ORDERKEY] [int] NOT NULL,
	[L_PARTKEY] [int] NOT NULL,
	[L_SUPPKEY] [int] NOT NULL,
	[L_LINENUMBER] [int] NOT NULL,
	[L_QUANTITY] [decimal](15, 2) NOT NULL,
	[L_EXTENDEDPRICE] [decimal](15, 2) NOT NULL,
	[L_DISCOUNT] [decimal](15, 2) NOT NULL,
	[L_TAX] [decimal](15, 2) NOT NULL,
	[L_RETURNFLAG] [char](1) NOT NULL,
	[L_LINESTATUS] [char](1) NOT NULL,
	[L_SHIPDATE] [date] NOT NULL,
	[L_COMMITDATE] [date] NOT NULL,
	[L_RECEIPTDATE] [date] NOT NULL,
	[L_SHIPINSTRUCT] [char](25) NOT NULL,
	[L_SHIPMODE] [char](10) NOT NULL,
	[L_COMMENT] [varchar](44) NOT NULL
"""

In [3]:
column_names = [line.split("]")[0][1:] for line in raw_schema.split("\n\t")[1:]]
column_names

['L_ORDERKEY',
 'L_PARTKEY',
 'L_SUPPKEY',
 'L_LINENUMBER',
 'L_QUANTITY',
 'L_EXTENDEDPRICE',
 'L_DISCOUNT',
 'L_TAX',
 'L_RETURNFLAG',
 'L_LINESTATUS',
 'L_SHIPDATE',
 'L_COMMITDATE',
 'L_RECEIPTDATE',
 'L_SHIPINSTRUCT',
 'L_SHIPMODE',
 'L_COMMENT']

In [14]:
from pyspark.sql.types import StructType, StructField, StringType
schema = StructType([StructField(column_name, StringType(), False) for column_name in column_names])

In [15]:
data = session.read\
              .format("csv")\
              .option("sep", "|")\
              .option("schema", schema)\
              .load("/home/orpington/dbgen-data/lineitem.tbl", schema=schema)

In [10]:
data.show(10)

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+-----------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|   L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+-----------------+----------+--------------------+
|         1|   155190|     7706|           1|        17|       21168.23|      0.04| 0.02|           N|           O|1996-03-13|  1996-02-12|   1996-03-22|DELIVER IN PERSON|     TRUCK|egular courts abo...|
|         1|    67310|     7311|           2|        36|       45983.16|      0.09| 0.06|           N|           O|1996-04-12|  1996-02-28|   1996-04-20| TAKE BACK RETURN|      MAIL|ly

In [53]:
data.count()

6001215

In [61]:
description = data.describe()


In [64]:
description.toPandas()

,summary,L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE,L_COMMENT
0,count,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215,6001215
1,mean,3000279.604204982,100017.98932999402,5000.602606138924,3.0005757167506912,25.507967136654827,38255.138484657095,0.049999430115268445,0.04001350893116897,None,None,None,None,None,None,None,None
2,stddev,1732187.8734803204,57735.69082650498,2886.961998730632,1.732431403651942,14.42626253701689,23300.43871096209,0.03161985510812605,0.02581655179884265,None,None,None,None,None,None,None,None
3,min,1,1,1,1,1,1000.00,0.00,0.00,A,F,1992-01-02,1992-01-31,1992-01-04,COLLECT COD,AIR,Tiresias
4,max,999975,99999,9999,7,9,99999.50,0.10,0.08,R,O,1998-12-01,1998-10-31,1998-12-31,TAKE BACK RETURN,TRUCK,zzle? slyly final platelets sleep quickly.


In [56]:
session.createDataFrame(data.rdd).printSchema()

root
 |-- L_ORDERKEY: string (nullable = true)
 |-- L_PARTKEY: string (nullable = true)
 |-- L_SUPPKEY: string (nullable = true)
 |-- L_LINENUMBER: string (nullable = true)
 |-- L_QUANTITY: string (nullable = true)
 |-- L_EXTENDEDPRICE: string (nullable = true)
 |-- L_DISCOUNT: string (nullable = true)
 |-- L_TAX: string (nullable = true)
 |-- L_RETURNFLAG: string (nullable = true)
 |-- L_LINESTATUS: string (nullable = true)
 |-- L_SHIPDATE: string (nullable = true)
 |-- L_COMMITDATE: string (nullable = true)
 |-- L_RECEIPTDATE: string (nullable = true)
 |-- L_SHIPINSTRUCT: string (nullable = true)
 |-- L_SHIPMODE: string (nullable = true)
 |-- L_COMMENT: string (nullable = true)



In [63]:
data.createOrReplaceTempView("lineitem")

In [71]:
from pyspark.sql.functions import count, lit
exprs = [count("*").alias("*")]
exprs += [count(lit(1)).alias("1")]
exprs += [count("L_SHIPMODE").alias("L_SHIPMODE")]
exprs += [count("L_RETURNFLAG").alias("L_RETURNFLAG")]
data.select(exprs).toPandas().T

,0
*,6001215
1,6001215
L_SHIPMODE,6001215
L_RETURNFLAG,6001215


In [74]:
session.sql("select count(*), count(distinct *) from lineitem").show()

+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|count(1)|count(DISTINCT L_ORDERKEY, L_PARTKEY, L_SUPPKEY, L_LINENUMBER, L_QUANTITY, L_EXTENDEDPRICE, L_DISCOUNT, L_TAX, L_RETURNFLAG, L_LINESTATUS, L_SHIPDATE, L_COMMITDATE, L_RECEIPTDATE, L_SHIPINSTRUCT, L_SHIPMODE, L_COMMENT)|
+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| 6001215|                                                                                                                                                                                                                   6001215|
+--------+----------------------------------------------------------------------

In [64]:
from pyspark.sql.functions import countDistinct
exprs = [countDistinct(column) for column in data.columns]
data.select(exprs).toPandas().T

,0
count(DISTINCT L_ORDERKEY),1500000
count(DISTINCT L_PARTKEY),200000
count(DISTINCT L_SUPPKEY),10000
count(DISTINCT L_LINENUMBER),7
count(DISTINCT L_QUANTITY),50
count(DISTINCT L_EXTENDEDPRICE),933900
count(DISTINCT L_DISCOUNT),11
count(DISTINCT L_TAX),9
count(DISTINCT L_RETURNFLAG),3
count(DISTINCT L_LINESTATUS),2


In [78]:
session.sql("""
select approx_count_distinct(L_COMMITDATE, 0.1) from lineitem limit 1000
""").show()

+-----------------------------------+
|approx_count_distinct(L_COMMITDATE)|
+-----------------------------------+
|                               2514|
+-----------------------------------+



In [82]:
10**-0

1

In [85]:
help(approx_count_distinct)

Help on function approx_count_distinct in module pyspark.sql.functions:

approx_count_distinct(col, rsd=None)
    Returns a new :class:`Column` for approximate distinct count of ``col``.
    
    >>> df.agg(approx_count_distinct(df.age).alias('c')).collect()
    [Row(c=2)]
    
    .. versionadded:: 2.1



In [106]:
data.cache()

DataFrame[L_ORDERKEY: string, L_PARTKEY: string, L_SUPPKEY: string, L_LINENUMBER: string, L_QUANTITY: string, L_EXTENDEDPRICE: string, L_DISCOUNT: string, L_TAX: string, L_RETURNFLAG: string, L_LINESTATUS: string, L_SHIPDATE: string, L_COMMITDATE: string, L_RECEIPTDATE: string, L_SHIPINSTRUCT: string, L_SHIPMODE: string, L_COMMENT: string]

In [31]:
from pyspark.sql.functions import approx_count_distinct, count, countDistinct

exprs = [countDistinct("L_COMMITDATE")]
exprs += [approx_count_distinct("L_COMMITDATE", 
                               rsd=float(0.30 - (exponent * 0.02)))
         for exponent 
         in range(10)]
data.select(exprs).toPandas().T

,0
count(DISTINCT L_COMMITDATE),2466
approx_count_distinct(L_COMMITDATE),2425
approx_count_distinct(L_COMMITDATE),2425
approx_count_distinct(L_COMMITDATE),2232
approx_count_distinct(L_COMMITDATE),2232
approx_count_distinct(L_COMMITDATE),2232
approx_count_distinct(L_COMMITDATE),2232
approx_count_distinct(L_COMMITDATE),2575
approx_count_distinct(L_COMMITDATE),2575
approx_count_distinct(L_COMMITDATE),2575


In [36]:
from pyspark.sql.functions import first, last
data.select(first("L_COMMITDATE"), 
            last("L_COMMITDATE"),
            first("L_COMMITDATE", True), 
            last("L_COMMITDATE", True)
           ).toPandas().T

,0
"first(L_COMMITDATE, false)",1996-02-12
"last(L_COMMITDATE, false)",1996-10-01
"first(L_COMMITDATE, true)",1996-02-12
"last(L_COMMITDATE, true)",1996-10-01


In [37]:
help(last)

Help on function last in module pyspark.sql.functions:

last(col, ignorenulls=False)
    Aggregate function: returns the last value in a group.
    
    The function by default returns the last values it sees. It will return the last non-null
    value it sees when ignoreNulls is set to true. If all values are null, then null is returned.
    
    .. versionadded:: 1.3



In [48]:
from pyspark.sql.functions import col, array, explode, sum, sumDistinct, max, min
session.range(10).select(explode(array([col("id")] * 3)).alias("nums"))\
                 .select(sum("nums"), sumDistinct("nums"), max("nums"), min("nums")).show()

+---------+------------------+---------+---------+
|sum(nums)|sum(DISTINCT nums)|max(nums)|min(nums)|
+---------+------------------+---------+---------+
|      135|                45|        9|        0|
+---------+------------------+---------+---------+



In [46]:
session.range(3).selectExpr("stack(3, id, id, id)").show()

+----+
|col0|
+----+
|   0|
|   0|
|   0|
|   1|
|   1|
|   1|
|   2|
|   2|
|   2|
+----+



In [54]:
from pyspark.sql.functions import (
    avg, mean, 
    var_pop, var_samp, stddev_pop, stddev_samp, 
    skewness, kurtosis
)
numeric_col = "L_QUANTITY"
data.select(avg(numeric_col), 
            mean(numeric_col),
            var_pop(numeric_col),
            var_samp(numeric_col),
            stddev_pop(numeric_col),
            stddev_samp(numeric_col),
            skewness(numeric_col),
            kurtosis(numeric_col)).toPandas().T

,0
avg(L_QUANTITY),25.507967
avg(L_QUANTITY),25.507967
var_pop(L_QUANTITY),208.117016
var_samp(L_QUANTITY),208.117051
stddev_pop(L_QUANTITY),14.426261
stddev_samp(L_QUANTITY),14.426263
skewness(L_QUANTITY),-0.001052
kurtosis(L_QUANTITY),-1.200236


In [56]:
from pyspark.sql.functions import corr, covar_samp, covar_pop

data.select([fun("L_QUANTITY", "L_DISCOUNT").alias(fun.func_name) 
             for fun in [corr, covar_samp, covar_pop]])\
    .toPandas()


,corr,covar_samp,covar_pop
0,-0.000752,-0.000343,-0.000343


In [59]:
from pyspark.sql.functions import collect_set, collect_list
session.range(10)\
       .selectExpr("stack(2, id, id) as id")\
       .select(collect_set("id"), collect_list("id"))\
       .toPandas()

,collect_set(id),collect_list(id)
0,"[0, 9, 1, 5, 2, 6, 3, 7, 4, 8]","[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, ..."


In [74]:
from tabulate import tabulate
result_df = data.groupby("L_SHIPINSTRUCT").count()
print(tabulate(result_df.sort(result_df.columns[1]).collect(), result_df.columns))
print
result_df = data.groupby("L_SHIPINSTRUCT", "L_LINESTATUS").count()
print(tabulate(result_df.sort(result_df.columns[2]).collect(), result_df.columns))

L_SHIPINSTRUCT       count
-----------------  -------
TAKE BACK RETURN   1499758
DELIVER IN PERSON  1500048
COLLECT COD        1500547
NONE               1500862

L_SHIPINSTRUCT     L_LINESTATUS      count
-----------------  --------------  -------
TAKE BACK RETURN   F                747938
NONE               F                749160
DELIVER IN PERSON  F                749531
COLLECT COD        F                749588
DELIVER IN PERSON  O                750517
COLLECT COD        O                750959
NONE               O                751702
TAKE BACK RETURN   O                751820


In [69]:
session.sql("""
  select L_SHIPINSTRUCT, 
         count(1) as counts 
    from lineitem 
group by L_SHIPINSTRUCT""").show()

+-----------------+-------+
|   L_SHIPINSTRUCT| counts|
+-----------------+-------+
| TAKE BACK RETURN|1499758|
|      COLLECT COD|1500547|
|             NONE|1500862|
|DELIVER IN PERSON|1500048|
+-----------------+-------+



In [7]:
session.range(5).rdd.flatMap(lambda x: x).sum()

10

In [9]:
import operator
dir(operator)

['__abs__',
 '__add__',
 '__and__',
 '__concat__',
 '__contains__',
 '__delitem__',
 '__delslice__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__ge__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__iadd__',
 '__iand__',
 '__iconcat__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__inv__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irepeat__',
 '__irshift__',
 '__isub__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__name__',
 '__ne__',
 '__neg__',
 '__not__',
 '__or__',
 '__package__',
 '__pos__',
 '__pow__',
 '__repeat__',
 '__rshift__',
 '__setitem__',
 '__setslice__',
 '__sub__',
 '__truediv__',
 '__xor__',
 '_compare_digest',
 'abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'delslice',
 'div',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'getslice',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'idiv',
 'ifloordiv',
 'ilshift',
 'imod',
 'imul',
 '

In [19]:
from pyspark.sql.functions import count, expr
data.groupby("L_SHIPINSTRUCT").agg(count("L_LINESTATUS"), expr("kurtosis(L_QUANTITY)")).show()

+-----------------+-------------------+--------------------+
|   L_SHIPINSTRUCT|count(L_LINESTATUS)|kurtosis(L_QUANTITY)|
+-----------------+-------------------+--------------------+
| TAKE BACK RETURN|            1499758| -1.1999482908232582|
|      COLLECT COD|            1500547| -1.2003436731194417|
|             NONE|            1500862| -1.2003951170092844|
|DELIVER IN PERSON|            1500048| -1.2002589048783696|
+-----------------+-------------------+--------------------+



In [27]:
from pyspark.sql.functions import count, expr
data.groupBy("L_SHIPINSTRUCT").agg({"L_LINESTATUS": "count", "*": "count"}).show()

+-----------------+--------+-------------------+
|   L_SHIPINSTRUCT|count(1)|count(L_LINESTATUS)|
+-----------------+--------+-------------------+
| TAKE BACK RETURN| 1499758|            1499758|
|      COLLECT COD| 1500547|            1500547|
|             NONE| 1500862|            1500862|
|DELIVER IN PERSON| 1500048|            1500048|
+-----------------+--------+-------------------+



In [42]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, max, col, dense_rank, rank

window_spec = Window.partitionBy("L_SHIPDATE")\
                    .orderBy(desc("L_ORDERKEY"))\
                    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

data.select("L_QUANTITY",
            "L_SHIPDATE", 
            "L_ORDERKEY",
            rank().over(window_spec).alias("rank"),
            dense_rank().over(window_spec).alias("dense_rank"),            
            max(col("L_QUANTITY").cast("float")).over(window_spec).alias("window")).show(100)

+----------+----------+----------+----+----------+------+
|L_QUANTITY|L_SHIPDATE|L_ORDERKEY|rank|dense_rank|window|
+----------+----------+----------+----+----------+------+
|        48|1992-05-01|    997317|   1|         1|  48.0|
|        27|1992-05-01|    990180|   2|         2|  48.0|
|        46|1992-05-01|    988676|   3|         3|  48.0|
|         4|1992-05-01|    987526|   4|         4|  48.0|
|        21|1992-05-01|    986884|   5|         5|  48.0|
|        45|1992-05-01|    986112|   6|         6|  48.0|
|         2|1992-05-01|      9793|   7|         7|  48.0|
|        40|1992-05-01|    978883|   8|         8|  48.0|
|        47|1992-05-01|    978533|   9|         9|  48.0|
|        31|1992-05-01|    977057|  10|        10|  48.0|
|        42|1992-05-01|    976006|  11|        11|  48.0|
|         2|1992-05-01|    975714|  12|        12|  48.0|
|        23|1992-05-01|    975269|  13|        13|  48.0|
|        45|1992-05-01|    975008|  14|        14|  48.0|
|         1|19

In [50]:
from pyspark.sql.functions import sum, expr

def do_summary(df):
data.rollup("L_SHIPINSTRUCT", "L_LINESTATUS")\
    .agg(sum(expr("cast(L_QUANTITY as decimal)")))\
    .selectExpr("L_SHIPINSTRUCT", "L_LINESTATUS", "`sum(CAST(L_QUANTITY AS DECIMAL(10,0)))` as total_quantity")\
    .sort("total_quantity")\
    .show()

+-----------------+------------+--------------+
|   L_SHIPINSTRUCT|L_LINESTATUS|total_quantity|
+-----------------+------------+--------------+
| TAKE BACK RETURN|           F|      19089435|
|             NONE|           F|      19104505|
|DELIVER IN PERSON|           F|      19114289|
|      COLLECT COD|           O|      19135219|
|      COLLECT COD|           F|      19137048|
|DELIVER IN PERSON|           O|      19149263|
|             NONE|           O|      19171510|
| TAKE BACK RETURN|           O|      19177526|
|DELIVER IN PERSON|        null|      38263552|
| TAKE BACK RETURN|        null|      38266961|
|      COLLECT COD|        null|      38272267|
|             NONE|        null|      38276015|
|             null|        null|     153078795|
+-----------------+------------+--------------+



In [55]:
from pyspark.sql import DataFrame
DataFrame.count(data)

6001215

In [57]:
from pyspark.sql.functions import sum, expr
from pyspark.sql import DataFrame
from tabulate import tabulate

def do_summary(summary_function):
    print(summary_function.func_name)
    df = summary_function(data, "L_SHIPINSTRUCT", "L_LINESTATUS")\
                            .agg(sum(expr("cast(L_QUANTITY as decimal)")))\
                            .selectExpr("L_SHIPINSTRUCT", "L_LINESTATUS", "`sum(CAST(L_QUANTITY AS DECIMAL(10,0)))` as total_quantity")\
                            .sort("total_quantity")
    print(tabulate(df.collect(), df.columns))
    print

do_summary(DataFrame.rollup)
do_summary(DataFrame.cube)

rollup
L_SHIPINSTRUCT     L_LINESTATUS      total_quantity
-----------------  --------------  ----------------
TAKE BACK RETURN   F                    1.90894e+07
NONE               F                    1.91045e+07
DELIVER IN PERSON  F                    1.91143e+07
COLLECT COD        O                    1.91352e+07
COLLECT COD        F                    1.9137e+07
DELIVER IN PERSON  O                    1.91493e+07
NONE               O                    1.91715e+07
TAKE BACK RETURN   O                    1.91775e+07
DELIVER IN PERSON                       3.82636e+07
TAKE BACK RETURN                        3.8267e+07
COLLECT COD                             3.82723e+07
NONE                                    3.8276e+07
                                        1.53079e+08

cube
L_SHIPINSTRUCT     L_LINESTATUS      total_quantity
-----------------  --------------  ----------------
TAKE BACK RETURN   F                    1.90894e+07
NONE               F                    1.91045e+07
DE

In [59]:
from pyspark.sql.functions import sum
data.groupby("L_SHIPINSTRUCT").pivot("L_LINESTATUS").agg(sum(col("L_QUANTITY").cast("decimal"))).show()

+-----------------+--------+--------+
|   L_SHIPINSTRUCT|       F|       O|
+-----------------+--------+--------+
| TAKE BACK RETURN|19089435|19177526|
|      COLLECT COD|19137048|19135219|
|             NONE|19104505|19171510|
|DELIVER IN PERSON|19114289|19149263|
+-----------------+--------+--------+



In [61]:
li = session.read.csv("/home/orpington/Desktop/dbgen/tpch-dbgen/lineitem.tbl*", sep="|")

In [62]:
li.count()

524715971